In [27]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import json

### Load agenda contents

In [28]:
f = urllib.request.urlopen('https://www.bredajazzfestival.nl/locatie/')
contents = f.read()
soup = BeautifulSoup(contents)

In [29]:
venues = soup.find_all(class_='event-content')
band_events = []

In [61]:
def place_location(name):
    if name == '01. Riant-Stoutenbeek podium - Grote Markt Midden': return {'lat': 51.588270, 'lon': 4.775947}
    if name == '02. Scorpions Security podium - Grote Markt zuid': return {'lat': 51.587545, 'lon': 4.776352}
    if name == '03. Meinselect podium - St Janstraat': return {'lat': 51.587659, 'lon': 4.777113}
    if name == '04. De Rooi Pannen podium - Pleintje Veemarktstraat': return {'lat': 51.588654, 'lon': 4.778927}
    if name == '05. Landgoed de Klokkenberg podium - Willem Merkxtuin': return {'lat': 51.589469, 'lon': 4.779095}
    if name == '06. Avans Talent podium - Kerkplein': return {'lat': 51.589274, 'lon': 4.774685}
    if name == '07. ’t Sas terras': return {'lat': 51.588489, 'lon': 4.774799}
    if name == '08. Van Iersel Luchtman Advocaten podium - \'t Sas Pleintje': return {'lat': 51.588308, 'lon': 4.774109}
    raise Exception('not supported: ' + name)

In [62]:
for v in venues:
    place_name = v.find('h3').text
    events = v.find_all(class_='each-event')
    for e in events:
        band_name = e.find('p').text
        event_url = e.find('a')['href']
        # print('>> EVENT:')
        # print(band_name)
        # print(event_url)
        # print(venue_name)
        band_events.append({
            'band_name': band_name,
            'event_url': event_url,
            'place_name': place_name,
            'event_location': place_location(place_name)
        })

Exception: not supported: 09. RB-Media podium - Havermarkt

### Add band details

In [46]:
def mon(name):
    if name == 'mei': return '05'
    raise Exception('not supported: ' + name)

In [32]:
for be in band_events:
    url = be['event_url']
    print('loading ' + url)
    f = urllib.request.urlopen(url)
    contents = f.read()
    soup = BeautifulSoup(contents)
    
    burl = soup.find('a', class_='artist-tag')
    if burl is not None:
        band_page_url = burl['href']
        be['band_page_url'] = band_page_url
    iurl = soup.find('img', alt='artist image')
    
    edate = soup.find('div', class_='date').find('span').text
    dparts = edate.split(' ')
        
    etime = soup.find('div', class_='time').find('span').text
    tparts = etime.split(' - ')

    be['event_start'] = dparts[2] + '-' + mon(dparts[1]) + '-' + dparts[0] + 'T' + tparts[0] + ':00'
    be['event_end'] = dparts[2] + '-' + mon(dparts[1]) + '-' + dparts[0] + 'T' + tparts[1] + ':00'
 
    if iurl is not None:
        band_image_url = iurl['src']
        be['band_image_url'] = band_image_url
        break

loading https://www.bredajazzfestival.nl/event/opening-breda-jazz-festival-2023/


### Show sample

In [33]:
pevents = pd.DataFrame(band_events)
pevents

,band_name,event_url,venue_name,event_start,event_end,band_image_url
0,Opening Breda Jazz Festival 2023,https://www.bredajazzfestival.nl/event/opening...,01. Riant-Stoutenbeek podium - Grote Markt Midden,2023-05-18T13:00:00,2023-05-18T13:30:00,https://www.bredajazzfestival.nl/wp-content/up...
1,BOSCO feat. Johnny Rosenberg,https://www.bredajazzfestival.nl/event/bosco-f...,01. Riant-Stoutenbeek podium - Grote Markt Midden,NaN,NaN,NaN
2,Calvin Johnson & Native Son,https://www.bredajazzfestival.nl/event/calvin-...,01. Riant-Stoutenbeek podium - Grote Markt Midden,NaN,NaN,NaN
3,Andrew Baham's New Orleans Tribute to Louis Ar...,https://www.bredajazzfestival.nl/event/andrew-...,01. Riant-Stoutenbeek podium - Grote Markt Midden,NaN,NaN,NaN
4,SOUL Brass Band,https://www.bredajazzfestival.nl/event/soul-br...,01. Riant-Stoutenbeek podium - Grote Markt Midden,NaN,NaN,NaN
...,...,...,...,...,...,...
207,LEMBO,https://www.bredajazzfestival.nl/event/lembo/,23. MotMot Gallery,NaN,NaN,NaN
208,Brass Bands From The World,https://www.bredajazzfestival.nl/event/brass-b...,Binnenstad,NaN,NaN,NaN
209,Brass Bands From The World,https://www.bredajazzfestival.nl/event/brass-b...,Binnenstad,NaN,NaN,NaN
210,Brass Bands From The World,https://www.bredajazzfestival.nl/event/brass-b...,Binnenstad,NaN,NaN,NaN


In [13]:
json_contents = json.dumps(band_events)

with open('output.json', 'w') as f:
    f.write(json_contents)